In [2]:
!pip install msgpack
!pip install -q h5py pyyaml 


    100% |████████████████████████████████| 143kB 271kB/s ta 0:00:01
  Running setup.py bdist_wheel for msgpack ... done
  Stored in directory: /Users/ianwong/Library/Caches/pip/wheels/f0/02/4c/525b56fce78c415eb8066f6554f9de02792df26b8f882f6d65
Successfully built msgpack
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
from __future__ import absolute_import, division,print_function
import os

import tensorflow as tf
from tensorflow import keras


In [8]:
(train_images, train_lables),(test_images,test_labels) = keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 5s 0us/step


In [9]:
train_labels = train_lables[:1000]
test_labels = test_labels[:1000]


test_images = test_images[:1000].reshape(-1,28*28)/255.0
train_images = train_images[:1000].reshape(-1, 28*28)/255.0

# define a model

In [11]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10,activation=tf.nn.softmax)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                 loss=keras.losses.sparse_categorical_crossentropy,
                 metrics=['accuracy'])
    return model

model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


# save checkpoits during training

In [12]:
checkpoint_path = './training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)
# create checkpoints callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1)
model = create_model()
model.fit(train_images,train_labels,
         epochs=10,
         validation_data=(test_images,test_labels),
         callbacks=[cp_callback])

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 1s 1ms/step - loss: 1.1585 - acc: 0.6490 - val_loss: 0.7008 - val_acc: 0.7890

Epoch 00001: saving model to ./training_1/cp.ckpt
Epoch 2/10
1000/1000 [==============================] - 0s 358us/step - loss: 0.4150 - acc: 0.8840 - val_loss: 0.5195 - val_acc: 0.8370

Epoch 00002: saving model to ./training_1/cp.ckpt
Epoch 3/10
1000/1000 [==============================] - 0s 389us/step - loss: 0.2836 - acc: 0.9270 - val_loss: 0.4310 - val_acc: 0.8700

Epoch 00003: saving model to ./training_1/cp.ckpt
Epoch 4/10
1000/1000 [==============================] - 0s 397us/step - loss: 0.1968 - acc: 0.9490 - val_loss: 0.4377 - val_acc: 0.8600

Epoch 00004: saving model to ./training_1/cp.ckpt
Epoch 5/10
1000/1000 [==============================] - 0s 403us/step - loss: 0.1485 - acc: 0.9710 - val_loss: 0.4121 - val_acc: 0.8650

Epoch 00005: saving model to ./training_1/cp.ckpt
Epoch 6/10
1000/100

In [13]:
!ls {checkpoint_dir}

checkpoint                  cp.ckpt.index
cp.ckpt.data-00000-of-00001


In [14]:
model = create_model()
loss, acc = model.evaluate(test_images,test_labels)
print('untrained model, accuracy:{:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 194us/step
untrained model, accuracy:11.70%


In [15]:
#load the weights from checkpoints
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images,test_labels)
print('restored model, accuracy:{:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 71us/step
restored model, accuracy:87.80%


In [16]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "./training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)

model = create_model()
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)



Epoch 00005: saving model to ./training_2/cp-0005.ckpt

Epoch 00010: saving model to ./training_2/cp-0010.ckpt

Epoch 00015: saving model to ./training_2/cp-0015.ckpt

Epoch 00020: saving model to ./training_2/cp-0020.ckpt

Epoch 00025: saving model to ./training_2/cp-0025.ckpt

Epoch 00030: saving model to ./training_2/cp-0030.ckpt

Epoch 00035: saving model to ./training_2/cp-0035.ckpt

Epoch 00040: saving model to ./training_2/cp-0040.ckpt

Epoch 00045: saving model to ./training_2/cp-0045.ckpt

Epoch 00050: saving model to ./training_2/cp-0050.ckpt


In [17]:
! ls {checkpoint_dir}

checkpoint                       cp-0040.ckpt.index
cp-0030.ckpt.data-00000-of-00001 cp-0045.ckpt.data-00000-of-00001
cp-0030.ckpt.index               cp-0045.ckpt.index
cp-0035.ckpt.data-00000-of-00001 cp-0050.ckpt.data-00000-of-00001
cp-0035.ckpt.index               cp-0050.ckpt.index
cp-0040.ckpt.data-00000-of-00001


In [18]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'./training_2/cp-0050.ckpt'

In [19]:
model = create_model()
model.load_weights(latest)
loss,acc = model.evaluate(test_images,test_labels)

print('restored model, accuracy:{:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 161us/step
restored model, accuracy:87.60%


In [20]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Restore the weights
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


1000/1000 [==============================] - 0s 181us/step
Restored model, accuracy: 87.60%
